In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

2023-11-09 22:14:20.664727: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 22:14:20.664796: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 22:14:20.664820: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 22:14:20.669880: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import numpy as np

filenames = ['./data/data1.csv', './data/data2.csv', './data/data3.csv']
arr = []
for filename in filenames:
    arr.append(np.loadtxt(filename, delimiter=",", dtype=np.float32))
    # print(arr[-1].shape)
arr = np.vstack(arr)

In [3]:
z_data, label = arr[:, 2], arr[:, 3]

In [4]:
window = lambda a, w, o: np.lib.stride_tricks.as_strided(a, strides = a.strides * 2, shape = (a.size - w + 1, w))[::o]
sampling_period = 5e-3
window_size, sliding_size = int(1/sampling_period), int(0.1/sampling_period)

In [5]:
inp_feat = window(z_data, window_size, sliding_size)
out_feat = np.sum(
    window(label, window_size, sliding_size),
    axis=1) > sliding_size//2
out_feat = out_feat.astype('float32')

print(inp_feat.shape, out_feat.shape)

(7636, 200) (7636,)


In [10]:
model = models.Sequential([
    layers.Dense(400, activation='relu', input_shape=(200,)),
    # layers.Dropout(0.3),
    layers.Dense(80, activation='relu'),
    # layers.Dropout(0.3),
    layers.Dense(16, activation='relu'),
    # layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # 輸出層
])

model.summary()

2023-11-09 22:14:22.146774: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-09 22:14:22.149768: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-09 22:14:22.149798: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-09 22:14:22.150971: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-09 22:14:22.151005: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 400)               80400     
                                                                 
 dense_1 (Dense)             (None, 80)                32080     
                                                                 
 dense_2 (Dense)             (None, 16)                1296      
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 113793 (444.50 KB)
Trainable params: 113793 (444.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
# 選擇損失函數和優化器
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),  
              metrics=['accuracy'])

# 定義 batch size
batch_size = 128

checkpoint_filepath = './models/tf_checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# 訓練模型
model.fit(x=inp_feat, y=out_feat, batch_size=batch_size, epochs=1000, shuffle=True, validation_split=0.2, callbacks=[model_checkpoint_callback])

# fit(
#     x=None,
#     y=None,
#     batch_size=None,
#     epochs=1,
#     verbose='auto',
#     callbacks=None,
#     validation_split=0.0,
#     validation_data=None,
#     shuffle=True,
#     class_weight=None,
#     sample_weight=None,
#     initial_epoch=0,
#     steps_per_epoch=None,
#     validation_steps=None,
#     validation_batch_size=None,
#     validation_freq=1,
#     max_queue_size=10,
#     workers=1,
#     use_multiprocessing=False
# )

Epoch 1/1000


2023-11-09 22:14:24.796078: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8c65dc9950 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-09 22:14:24.796119: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-11-09 22:14:24.799809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-09 22:14:24.926420: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2023-11-09 22:14:24.996994: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


48/48 [==============================] - 3s 8ms/step - loss: 1.4585 - accuracy: 0.5270 - val_loss: 0.9933 - val_accuracy: 0.5556
Epoch 2/1000
48/48 [==============================] - 0s 7ms/step - loss: 0.7376 - accuracy: 0.6737 - val_loss: 0.9113 - val_accuracy: 0.5720
Epoch 3/1000
48/48 [==============================] - 0s 7ms/step - loss: 0.7060 - accuracy: 0.6785 - val_loss: 0.8844 - val_accuracy: 0.5746
Epoch 4/1000
48/48 [==============================] - 0s 5ms/step - loss: 0.6829 - accuracy: 0.6835 - val_loss: 0.8708 - val_accuracy: 0.5825
Epoch 5/1000
48/48 [==============================] - 0s 6ms/step - loss: 0.6646 - accuracy: 0.6863 - val_loss: 0.8422 - val_accuracy: 0.5897
Epoch 6/1000
48/48 [==============================] - 0s 6ms/step - loss: 0.6464 - accuracy: 0.6917 - val_loss: 0.8307 - val_accuracy: 0.5942
Epoch 7/1000
48/48 [==============================] - 0s 6ms/step - loss: 0.6292 - accuracy: 0.6922 - val_loss: 0.8129 - val_accuracy: 0.5949
Epoch 8/1000
48/48 

In [12]:
model.load_weights(checkpoint_filepath)
model.save('./models/model.h5')
# model.predict()

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
import keras
keras.models.load_model('./models/model.h5')

In [14]:
tf.__version__

'2.14.0'